# Modélisation

## Choix du modèle 

- Modèle de régression : détermination de la note du vin (0 -> 10)
- Modèle de classification : classer les bons vins des mauvais

> _Objectif_ : Orienter le projet vers une fonction précise, et adapter le modèle à cet objectif

- _Remarque_ : Pour cette exercice, nous réaliserons un régression.

In [5]:
# Chargement des bibliothèques et des données
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib

X_train, X_test, y_train, y_test = joblib.load("../data/processed/splits.pkl")